In [65]:
# Import statements
import urllib
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
from dateutil.parser import parse
from dateutil import parser
from collections import defaultdict,Counter
import re
import pickle
import os
import pandas as pd
import numpy as np
import glob
import datetime
import json

os.chdir('/Users/yiweiluo/scientific-debates')
from utils import get_fname,fulltext_exists#,soupify
os.chdir('./1_data_scraping')

In [11]:
def get_google_res_stance(x):
    if 'foxnews.com' in x:
        return 'anti'
    elif 'breitbart.com' in x:
        return 'anti'
    elif 'blaze.com' in x:
        return 'anti'
    elif 'pjmedia.com' in x:
        return 'anti'
    elif 'nationalreview.com' in x:
        return 'anti'
    elif 'dailycaller.com' in x:
        return 'anti'
    elif 'reason.com' in x:
        return 'anti'
    elif 'americanthinker.com' in x:
        return 'anti'
    elif 'redstate.com' in x:
        return 'anti'
    elif 'infowars.com' in x:
        return 'anti'
    else:
        return 'pro'

In [12]:
NYT_SECTIONS_TO_REMOVE = set(['/automobiles/','/autoreviews/','/autoshow/','/business/','/campaign-stops/',
                          '/crosswords/',
               '/booming/','/giving/','/gmcvb/','/jobs/','/lens/','/letters/','/newyorktoday/',
               '/nutrition/','/sept-11-reckoning/','/smallbusiness/',
               '/sunday-review/','/garden/','/arts/','/theater/','/sports/','/dining/','/books/','/weekinreview/','/your-money/',
                         '/movies/','/fashion/','/technology/','/pageoneplus/','/travel/','/nytnow/',
                         '/public-editor/','/education/','/learning/','/podcasts/','/style/','/t-magazine/',
                         '/reader-center/','/awardsseason/','/briefing/','/dealbook/','/es/',
                          '/greathomesanddestinations/','/interactive/','/media/',
                         '/mutfund/','/obituaries/','/personaltech/','/realestate/',
                          '/smarter-living/','/todayspaper/','/your-money/','/yourtaxes/',
                             '/slideshow/','/interactive/'])

In [13]:
BLACKLIST_URL_STRS = set(['/tag/','/author/','/clips/','/podcasts/','/subject/','/authors/',
                         '/category/','/person/','/category/','/shows/','/video/','/topic/',
                         '/es/','/topics/','/de/','/tags/','/slideshow/',
                         '/interactive/','/transcripts/','/headlines/'])

In [14]:
BLACKLIST_URL_INIT_STRS = set(['rss.','feeds.','rssfeeds.'])

In [98]:
def is_rss(url):
    for xx in BLACKLIST_URL_INIT_STRS:
        if url[:len(xx)] == xx:
            return True
    return False

def is_blacklist(url):
    for xx in BLACKLIST_URL_STRS:
        if xx in url:
            return True
    return False

from urllib.parse import urlparse

def get_hostname(url, uri_type='both'):
    """Get the host name from the url"""
    parsed_uri = urlparse(url)
    if uri_type == 'both':
        return '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    elif uri_type == 'netloc_only':
        return '{uri.netloc}'.format(uri=parsed_uri)

def strip_url(url):
    if url.startswith('http'):
        url = re.sub(r'https?://', '', url)
        #print(url)
    if url.startswith('www.'):
        url = re.sub(r'www.', '', url)
    return url

from urllib.error import URLError

def soupify(url):
        
    try:
        req = urllib.request.Request(url, headers={'User-Agent' : "Magic Browser"}) 
        con = urllib.request.urlopen( req )
        html = con.read()
        soup = BeautifulSoup(html,'html.parser')
        return soup
    except (URLError,TimeoutError) as e:
        return None

# Load data

In [10]:
google_cc_urls = pickle.load(open('google_search_res_climate_change.pkl','rb')) # domain, keyword, title, url, date
new_anti_google_cc_urls = pickle.load(open('new_anti_google_search_res_climate_change.pkl','rb'))
R_google_cc_urls = pickle.load(open('R_google_search_res_climate_change.pkl','rb'))
mediacloud_cc_urls = pd.read_pickle('mediacloud_df.pkl') # ap_syndicated, domain, title, url, date
reid_cc_urls = pd.read_pickle('reid_urls.pkl') # url only
prev_urls = pd.read_pickle('filtered_all_url_df.pkl')

In [16]:
filtered_urls = []
filtered_titles = []
filtered_dates = []
filtered_domains = []
filtered_stances = []
filtered_topics = []
filtered_is_AP = []

In [17]:
for key in google_cc_urls:
    for keyword in google_cc_urls[key]:
        for item in google_cc_urls[key][keyword]:
            url = strip_url(item[1])
            if not is_rss(url) and not is_blacklist(url):
                title = item[0]
                date = item[2] if len(item) > 2 else None
                stance = get_google_res_stance(url)
                topic = 'cc'
                is_AP = None

                if ' | ' not in title:
                    filtered_urls.append(url)
                    filtered_titles.append(title)
                    filtered_dates.append(date)
                    filtered_domains.append(key)
                    filtered_stances.append(stance)
                    filtered_topics.append(topic)
                    filtered_is_AP.append(is_AP)
                    
for key in new_anti_google_cc_urls:
    for keyword in new_anti_google_cc_urls[key]:
        for item in new_anti_google_cc_urls[key][keyword]:
            url = strip_url(item[1])
            if not is_rss(url) and not is_blacklist(url):
                title = item[0]
                date = item[2] if len(item) > 2 else None
                stance = get_google_res_stance(url)
                topic = 'cc'
                is_AP = None

                if ' | ' not in title:
                    filtered_urls.append(url)
                    filtered_titles.append(title)
                    filtered_dates.append(date)
                    filtered_domains.append(key)
                    filtered_stances.append(stance)
                    filtered_topics.append(topic)
                    filtered_is_AP.append(is_AP)
                    
for key in R_google_cc_urls:
    for keyword in R_google_cc_urls[key]:
        for item in R_google_cc_urls[key][keyword]:
            url = strip_url(item[1])
            if not is_rss(url) and not is_blacklist(url):
                title = item[0]
                date = item[2] if len(item) > 2 else None
                stance = 'anti'#get_google_res_stance(url)
                topic = 'cc'
                is_AP = None

                if ' | ' not in title:
                    filtered_urls.append(url)
                    filtered_titles.append(title)
                    filtered_dates.append(date)
                    filtered_domains.append(key)
                    filtered_stances.append(stance)
                    filtered_topics.append(topic)
                    filtered_is_AP.append(is_AP)

In [18]:
for ix in mediacloud_cc_urls.index:
    row = mediacloud_cc_urls.loc[ix]
    url = strip_url(row['url']) if 'http' in row['url'] else strip_url(row['guid'])
    if not is_rss(url) and not is_blacklist(url):
        title = row['clean_title']
        date = row['publish_date']
        domain = row['media_name']
        stance = row['stance']
        topic = row['topic']
        is_AP = row['ap_syndicated']

        if ' | ' not in title:
            filtered_urls.append(url)
            filtered_titles.append(title)
            filtered_dates.append(date)
            filtered_domains.append(domain)
            filtered_stances.append(stance)
            filtered_topics.append(topic)
            filtered_is_AP.append(is_AP)

In [19]:
for key in reid_cc_urls:
    for url in reid_cc_urls[key][0].values:
        url = strip_url(url)
        if not is_rss(url) and not is_blacklist(url):
            filtered_urls.append(url)
            filtered_titles.append(None)
            filtered_dates.append(None)
            filtered_domains.append(key)
            filtered_stances.append('pro')
            filtered_topics.append('cc')
            filtered_is_AP.append(None)

In [20]:
prev_cc_urls = prev_urls.loc[prev_urls['topic'] == 'cc']
prev_cc_urls['is_AP'] = [None]*len(prev_cc_urls)
prev_cc_urls.shape

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(17475, 9)

In [21]:
combined_df = pd.DataFrame({'url':filtered_urls,
                              'title':filtered_titles,
                              'date':filtered_dates,
                              'domain':filtered_domains,
                              'stance':filtered_stances,
                              'topic':filtered_topics,
                              'is_AP':filtered_is_AP})

In [22]:
combined_df = combined_df.append(prev_cc_urls[combined_df.columns],
                                 ignore_index=True)
combined_df.shape

(98892, 7)

# Check consistency of coding

In [164]:
def standardize_domain(x):
    if x == 'Guardian US':
        return 'guardian_us'
    elif x == 'the_boston_globe':
        return 'boston_globe'
    elif 'washingtonpost.com' in x:
        return 'wapo'
    elif 'vox.com' in x:
        return 'vox'
    elif 'breitbart.com' in x:
        return 'breitbart'
    elif 'nytimes.com' in x:
        return 'nyt'
    elif 'motherjones.com' in x:
        return 'mj'
    elif x == 'democracy_now':
        return 'dem_now'
    elif 'foxnews.com' in x:
        return 'fox'
    elif 'buzzfeednews.com' in x or 'www.buzzfeed' in x:
        return 'buzzfeed'
    elif 'https://childrenshealthdefense.org/' in x:
        return 'chd'
    elif x == 'Daily Caller' or 'www.dailycaller' in x:
        return 'daily_caller'
    elif 'www.dailysignal' in x:
        return 'daily_signal'
    elif x == 'Washington Post':
        return 'wapo'
    elif 'theblaze.com' in x or x == 'the_blaze':
        return 'blaze'
    elif 'democracynow.org' in x:
        return 'dem_now'
    elif x == 'Grist':
        return 'grist'
    elif x == 'New York Times':
        return 'nyt'
    elif 'nationalreview.com' in x:
        return 'nat_review'
    elif 'thenation.com' in x:
        return 'nation'
    elif x == 'Breitbart':
        return 'breitbart'
    elif x == 'Christian Science Monitor':
        return 'cs_monitor'
    elif 'https://www.csmonitor/' in x:
        return 'cs_monitor'
    elif x == 'buzzfeed_news':
        return 'buzzfeed'
    elif x == 'washington_post':
        return 'wapo'
    elif x == 'FOX News':
        return 'fox'
    elif x == 'USA Today':
        return 'usa_today'
    elif x == 'Mother Jones':
        return 'mj'
    elif x == 'NBC News' or 'nbcnews.com' in x:
        return 'nbc'
    elif x == 'Democracy Now!':
        return 'dem_now'
    elif x == 'National Review':
        return 'nat_review'
    elif x == 'CNS News':
        return 'cns'
    elif x == 'Buzzfeed':
        return 'buzzfeed'
    elif x == 'The Nation':
        return 'nation'
    elif 'pjmedia' in x:
        return 'pj'
    elif 'pajamas_media' in x:
        return 'pj'
    elif x == 'pj' or x == 'pjmedia':
        return 'pj'
    elif 'www.americanthinker' in x:
        return 'american_thinker'
    elif 'www.redstate' in x:
        return 'redstate'
    elif 'www.infowars' in x:
        return 'infowars'
    elif 'www.wnd' in x:
        return 'wnd'
    elif 'www.nysun' in x:
        return 'new_york_sun'
    elif 'www.cnsnews' in x:
        return 'cns'
    elif 'www.realclearpolitics' in x:
        return 'real_clear_politics'
    elif 'www.newsmax' in x:
        return 'newsmax'
    elif 'www.newsbusters.org' in x:
        return 'newsbusters'
    elif 'www.unionleader' in x:
        return 'unionleader'
    elif 'www.townhall' in x:
        return 'townhall'
    elif 'www.hotair' in x:
        return 'hot_air'
    else:
        return x.lower().strip().replace(' ','_').replace('.com','')

In [165]:
combined_df['domain'] = combined_df.domain.apply(standardize_domain)
combined_df['domain'] = combined_df.domain.apply(standardize_domain) # do twice

In [157]:
combined_df.title = combined_df.title.apply(lambda x: x.strip().lower() if x 
                                           is not None else x)

In [158]:
combined_df = combined_df.loc[~pd.isnull(combined_df.stance)]
combined_df.shape

(85112, 8)

In [28]:
stance_reg_dict = {'l':'pro','r':'anti','c':'between','pro':'pro','anti':'anti','between':'between'}
combined_df['stance'] = combined_df.stance.apply(lambda x: stance_reg_dict[x])
combined_df.stance.value_counts()

pro        57971
anti       37918
between     2411
Name: stance, dtype: int64

In [29]:
def standardize_date(x):
    if x is not None:
        if type(x) == str:
            x = x.replace('·','').strip()
            try:
                return parser.parse(x)
            except ValueError:
                #print(x)
                return None
        elif type(x) == datetime.datetime:
            return x
        else:
            return x.to_pydatetime()

In [34]:
combined_df['date'] = combined_df['date'].apply(standardize_date)
Counter([type(x) for x in combined_df.date])

Counter({datetime.datetime: 83658, NoneType: 14642})

In [36]:
14642+83658

98300

# Dedup

In [35]:
combined_df = combined_df.sort_values(['title'],axis=0)#,ignore_index=True)
combined_df.shape

(98300, 7)

In [37]:
combined_df = combined_df.drop_duplicates(subset='url',keep='first')#,ignore_index=True)
print(combined_df.shape)
combined_df.to_pickle('temp_combined_df.pkl')

(85112, 7)


In [166]:
combined_df.to_pickle('temp_combined_df.pkl')

# Scrape full text, title, date

In [84]:
from newspaper import Article
from newspaper import ArticleException
from urllib.error import HTTPError
from date_guesser import guess_date, Accuracy

In [150]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def get_guess_date(url):
    req = urllib.request.Request(url, headers={'User-Agent' : "Magic Browser"}) 
    try:
        con = urllib.request.urlopen( req )
        html = con.read()
        guess = guess_date(url=url,
                           html=html)
        return guess.date
    except (HTTPError,TimeoutError,URLError) as e:
        return None

In [108]:
def newspaper_parse(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return (article.title,article.publish_date,
                article.text.replace('\n',' '))
    except ArticleException:
        return (None,None,None)

In [40]:
from nltk.tokenize import sent_tokenize

def get_fulltext(url,domain):
    scraping_url = 'http://'+url
    
    stop_ix,title,text = None,None,None
    if domain == 'alternet':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'american_conservative':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -1
    elif domain == 'activistpost':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -4
    elif domain == 'american_thinker':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -1
    elif domain == 'the_atlanta_journal_constitution':
        soup = soupify(scraping_url)
        if soup is not None:
            ps = soup.find('div',attrs={'class':'story-text clearfix'}).find_all('p')
            text = ' '.join([p.text.replace('\n',' ') for p in ps])
            title = soup.find('div',attrs={'class':'tease__text'}).find('h1').text.strip()
            stop_ix = -5
    elif domain == 'bipartisanreport':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'blaze':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'boston_globe':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'breitbart':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'buzzfeed':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -1
    elif domain == 'cbn':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'cbs_news':
        soup = soupify(scraping_url)
        ps = soup.find('section',attrs={'class':'content__body'}).find_all('p')
        text = ' '.join([p.text.replace('\n',' ') for p in ps])
        title = soup.find('header',attrs={'class':'content__header'}).find('h1').text.strip()
    elif domain == 'charismanews':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -14
    elif domain == 'chd':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'https://www.citizens.org/':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'cns':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'commdiginews':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'conservative_review':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'conservative_treehouse':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'conservativedailynews':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -4
    elif domain == 'conservativefiringline':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -10
    elif domain == 'cs_monitor':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -1
    elif domain == 'daily_caller':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -3
    elif domain == 'daily_dot':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'www.dailysignal':
        soup = soupify(scraping_url)
        if soup is not None:
            title = soup.find('h1').text.strip()
            try:
                ps = soup.find('div',attrs={'class':'tds-content'}).find_all('p')
            except AttributeError:
                ps = soup.find('div',attrs={'class':'amp-wp-article-content'}).find_all('p')
            text = ' '.join([p.text.replace('\n',' ') for p in ps])
    elif domain == 'dallas_morning_news':
        soup = soupify(scraping_url)
        if soup is not None:
            title = soup.find('h1').text.strip()
            ps = soup.find('div',attrs={'itemprop':'articleBody'})
            text = ' '.join([p.text.replace('\n',' ') for p in ps])
    elif domain == 'dem_now':
        soup = soupify(scraping_url)
        try:
            ps = soup.find('div',attrs={'itemprop':'articleBody'}).find_all('p')
        except AttributeError:
            ps = soup.find('div',attrs={'class':'story_summary'}).find_all('p')
        text = ' '.join([p.text.replace('\n', ' ') for p in ps])
    elif domain == 'drudgereport':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'forbes':
        soup = soupify(scraping_url)
        if soup is not None:
            try:
                ps = soup.find('div',attrs={'class':'article-body fs-article fs-responsive-text current-article'}).find_all('p')
                text = ' '.join([p.text.replace('\n', ' ') for p in ps])
                title = soup.find('h1').text.strip()
            except AttributeError:
                pass
    elif domain == 'fox':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'gateway_pundit':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'gawker':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -1
    elif domain == 'grabien':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'grist':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'guardian_us':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'hot_air':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'https://adultvaccinesnow.org/blog/':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'https://immunizationevidence.org/featured_issues/':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'https://shotofprevention/':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -1
    elif domain == 'https://www.voicesforvaccines.org/blog/':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'https://icandecide.org/':
        pass
    elif domain == 'independentsentinel':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'infowars':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'inthesetimes':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'libertyunyielding':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -1
    elif domain == 'mj':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'nat_review':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'nation':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'nbc':
        soup = soupify(scraping_url)
        ps = soup.find('div',attrs={'class':'article-body__content'}).\
        find_all('p',attrs={'class':'endmarkEnabled'})
        text = ' '.join([p.text.replace('\n',' ') for p in ps])
        title = soup.find('h1').text.strip()
    elif domain == 'msnbc':
        soup = soupify(scraping_url)
        ps = soup.find('div',attrs={'itemprop':'articleBody'}).find_all('p')
        text = ' '.join([p.text.replace('\n',' ') for p in ps])
        title = soup.find('h1').text.strip()
    elif domain == 'new_york_magazine':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'new_york_sun':
        soup = soupify(scraping_url)
        if soup is not None:
            ps = soup.find('div',attrs={'itemprop':'articleBody'}).find_all('p')
            text = ' '.join([p.text.replace('\n',' ') for p in ps])
            title = soup.find('h1').text.strip()
    elif domain == 'newsbusters':
        soup = soupify(scraping_url)
        if soup is not None:
            ps = soup.find('div',attrs={'class':'field-item'}).find_all('p')
            text = ' '.join([p.text.replace('\n',' ') for p in ps])
            title = soup.find('h2').text.strip()
    elif domain == 'www.newsbusters.org':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'newsday':
        soup = soupify(scraping_url)
        if soup is not None:
            title = soup.find('div',attrs={'class':'sticky'}).find('header').find('h1').text.strip()
            ps = soup.find('div',attrs={'id':'contentAccess'}).find_all('p')
            text = ' '.join([p.text.replace('\n',' ') for p in ps])
    elif domain == 'newsmax':
        soup = soupify(scraping_url)
        if soup is not None:
            title = soup.find('h1',attrs={'itemprop':'headline'}).text.strip()
            text = soup.find('div',attrs={'id':'mainArticleDiv'}).text.replace('\n',' ')
            #text = ' '.join([p.text.replace('\n',' ') for p in ps])
    elif domain == 'www.newsmax':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'newsweek':
        soup = soupify(scraping_url)
        ps = soup.find('div',attrs={'class':'article-content'}).find_all('p')
        text = ' '.join([p.text.replace('\n', ' ') for p in ps])
    elif domain == 'newswithviews':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -10
    elif domain == 'ny_post':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'nyt':
        try:
            soup = soupify(scraping_url)
            if soup is not None:
                ps = soup.find('section',attrs={'itemprop':'articleBody'}).find_all('p')#,recursive=False)
                text = ' '.join([p.text.replace('\n', ' ') for p in ps])
                stop_ix = -5
        except HTTPError:
            pass
    elif domain == 'pajamas_media':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'patriotpost.us':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'pj':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'https://physiciansforinformedconsent.org/':
        soup = soupify(scraping_url)
        ps = soup.find('div',attrs={'class':'entry-content'}).find_all('p',attrs={'class':'responsiveNews'})
        text = ' '.join([p.text.replace('\n', ' ') for p in ps])
    elif domain == 'progressivestoday':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -1
    elif domain == 'quartz':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'rare.us':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'www.realclearpolitics':
        soup = soupify(scraping_url)
        #print('soup:',soup)
        try:
            ps = soup.find('div',attrs={'class':'article_body'}).find_all('p',recursive=False)
            print('ps:',ps)
            text = ' '.join([p.text.replace('\n',' ') for p in ps])
        except AttributeError:
            try:
                ps = soup.find('div',attrs={'id':'alpha'}).find_all('p')
                text = ' '.join([p.text.replace('\n',' ') for p in ps])
            except AttributeError:
                pass
        
        try:
            title = soup.find('h2').text.strip()
        except AttributeError:
            pass
    elif domain == 'real_clear_politics':
        soup = soupify(scraping_url)
        if soup is not None:
            ps = soup.find('div',attrs={'class':'article-body-text'}).find_all('p')
            text = ' '.join([p.text.replace('\n',' ') for p in ps])
            title = soup.find('h1').text.strip()

    elif domain == 'reason':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -3
    elif domain == 'redstate':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -1
    elif domain == 'sgtreport':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -1
    elif domain == 'shoebat':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -1
    elif domain == 'sonsoflibertymedia':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -1
    elif domain == 'the_american_conservative':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'the_american_spectator':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'the_columbus_dispatch':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'the_nation':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'the_progressive':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'therealnews':
        soup = soupify(scraping_url)
        if soup is not None:
            title = soup.find('h1').text.strip()
            try:
                ps = soup.find('div',attrs={'id':'column-content'}).find_all('p')
            except AttributeError:
                ps = soup.find('div',attrs={'class':'fl-rich-text'}).find_all('p')
            text = ' '.join([p.text.replace('\n',' ') for p in ps])
    elif domain == 'the_verge':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'the_week':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'think_progress':
        soup = soupify(scraping_url)
        if soup is not None:
            title = soup.find('h1').text.strip()
            ps = soup.find('div',attrs={'class':'post__content'}).find_all('p')
            text = ' '.join([p.text.replace('\n',' ') for p in ps])
    elif domain == 'townhall':
        soup = soupify(scraping_url)
        if soup is not None:
            title = soup.find('h1').text.strip()
            ps = soup.find('section',attrs={'id':'article-body'}).find_all('p',recursive=False)
            text = ' '.join([p.text.replace('\n',' ') for p in ps])
    elif domain == 'unionleader':
        pass
    elif domain == 'usa_today':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'vax_safety_commission':
        root_url = 'https://vaccinesafetycommission.org/studies.html'
        soup = soupify(root_url)
        panel_bodies = soup.find_all('div',attrs={'class':'panel-body'})
        #print(len(panel_bodies))
        text = ""
        for pb in panel_bodies:
            text += pb.text.strip()
    elif domain == 'vice':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'https://www.voicesforvaccines.org/blog/':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'vox':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'washington_times':
        title,text = newspaper_parse(scraping_url)
    elif domain == 'wapo':
        title,text = newspaper_parse(scraping_url)
        stop_ix = -2
    elif domain == 'wnd':
        soup = soupify(scraping_url)
        if soup is not None:
            title = soup.find('h1').text.strip()
            ps = soup.find('div',attrs={'class':'entry-content'}).find_all('p')
            text = ' '.join([p.text.replace('\n',' ') for p in ps])
    else:
        print('Unknown domain:',domain)
        with open('unknown_domains.txt','w') as f:
            f.write(domain+'\n')

    if text is not None and len(text) > 0:
        text = text.strip()
        sent_tokens = sent_tokenize(text)

        # Remove final few sentences (usually about social media)
        sent_tokens = sent_tokens[:stop_ix]
        text = ' '.join(sent_tokens)

    return (title,text)

In [143]:
def guess_title(url):
    try:
        soup = soupify(url)
    except TimeoutError:
        soup = None
    if soup is not None:
        titles = []
        h1 = soup.find('h1')
        h2 = soup.find('h2')
        if h1 is not None:
            titles.append(h1.text.strip())
        if h2 is not None:
            titles.append(h2.text.strip())
        if len(titles) > 0:
            return titles
        else:
            return None
    return None

In [41]:
combined_df.domain.value_counts()

nyt             9363
mj              3894
breitbart       3161
wapo            2920
fox             2845
                ... 
azcentral          8
cbn                6
cincinnati         5
charismanews       3
bgr                1
Name: domain, Length: 132, dtype: int64

In [57]:
combined_df.shape

(85112, 7)

In [159]:
combined_df.reset_index(drop=True,inplace=True)
combined_df.index

RangeIndex(start=0, stop=85112, step=1)

In [160]:
combined_df['guid'] = ['url_no_{}'.format(i) for i in range(len(combined_df))]
#combined_df

In [113]:
existing_guids_set = set(glob.glob('url_meta/*.json'))
missing_guids = [x for x in combined_df.guid.values[:last_ix] 
                 if 'url_meta/{}.json'.format(x) not in existing_guids_set]
print(len(existing_guids_set),len(missing_guids))

24174
7025


In [116]:
missing_guids_df = combined_df.loc[combined_df.guid.isin(missing_guids)]
missing_guids_df.index,missing_guids_df.shape

(Int64Index([   80,   152,   266,   289,   294,   302,   315,   317,   321,
               327,
             ...
             31154, 31155, 31163, 31167, 31169, 31170, 31171, 31176, 31177,
             31192],
            dtype='int64', length=7025), (7025, 8))

In [ ]:
for ix in range(1872,len(missing_guids_df)):
    row = missing_guids_df.iloc[ix]
    url = row['url']
    scrape_url = 'https://'+url
    res = newspaper_parse(scrape_url)
    #if res[0] is not None:
    title = res[0]
    date = res[1]
    if title is None:
        title = guess_title(scrape_url)
    if date is None:
        date = get_guess_date(scrape_url)
    if title is None and date is None:
        pass
    else:
        json_name = row['guid']
        with open(os.path.join('url_meta',json_name+'.json'),'w') as f:
            json_object = json.dumps({
                    "title":title,
                    "date":date.isoformat() if date is not None else None,
                    "text":res[2]
            }, indent = 4) 
            f.write(json_object)
    if ix % 100 == 0:
        print(ix)
        print(url)
        existing_guids_set = set(glob.glob('url_meta/*.json'))
        print('Length existing:',len(existing_guids_set))
        missing_guids = [x for x in combined_df.guid.values[:last_ix] 
                         if 'url_meta/{}.json'.format(x) not in existing_guids_set]
        print('Length missing:',len(missing_guids))

1900
dailykos.com/story/2014/03/09/1283109/-Book-review-Capitalism-and-climate-change
Length existing: 24985
Length missing: 6214
2000
apnews.myway.com/article/20150116/us-sci--hottest_year-b64ea00652.html
Length existing: 25011
Length missing: 6188
2100
startribune.com/nation/331193571.html
Length existing: 25033
Length missing: 6166
2200
sacbee.com/news/local/transportation/article236925038.html#storylink=rss
Length existing: 25039
Length missing: 6160
2300
kansascity.com/2011/02/15/2656343/canada-pipeline-deal-too-costly.html#storylink=rss
Length existing: 25076
Length missing: 6123
2400
chicago.suntimes.com/environment/william-nordhaus-donald-trump-carbon-tax-renewed-attention-climate-change/
Length existing: 25092
Length missing: 6107
2500
sacbee.com/2013/04/08/5324980/chiles-codelco-begins-exports.html#mi_rss=Wire%20World%20News
Length existing: 25128
Length missing: 6071
2600
forbes.com/sites/stratfor/2014/08/19/chinas-shale-production-falls-short-but-goals-remain-in-place/
Leng

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


3000
thinkprogress.org.feedsportal.com/c/34726/f/638927/s/2765d615/l/0Lthinkprogress0Borg0Cclimate0C20A130C0A10C10A0C14210A310Cclimate0Einformed0Edevelopment0Ehow0Eto0Erebuild0Ein0Ea0Ewarming0Eworld0C/story01.htm
Length existing: 25290
Length missing: 5909
3100
forbes.com/sites/kensilverstein/2013/10/05/political-discord-heightens-as-coal-sector-revels-in-latest-conquest/
Length existing: 25301
Length missing: 5898
3200
neighbors.denverpost.com/viewtopic.php?p=2973013#p2973013
Length existing: 25311
Length missing: 5888
3300
neighbors.denverpost.com/viewtopic.php?p=2780872#p2780872
Length existing: 25311
Length missing: 5888
3400
startribune.com/bios/441394853.html
Length existing: 25323
Length missing: 5876
3500
sun-sentinel.com/fl-pollution-crackdown-palm-20151030-story.html
Length existing: 25366
Length missing: 5833
3600
chicagotribune.com/ct-dakota-access-protest-service-preparation-20170328-story.html
Length existing: 25386
Length missing: 5813
3700
therealnews.com/idirect.php?i=

In [177]:
ix,scrape_url

(1871,
 'https://midwestdemocracy.com/articles/blog-watch-friday-may-11/#storylink=rss')

In [152]:
get_guess_date(scrape_url)

In [135]:
article = Article(scrape_url)

In [136]:
article.download()

In [138]:
soup = soupify(scrape_url)

TimeoutError: [Errno 60] Operation timed out

In [140]:
req = urllib.request.Request(scrape_url, headers={'User-Agent' : "Magic Browser"}) 
con = urllib.request.urlopen( req )
html = con.read()

TimeoutError: [Errno 60] Operation timed out

In [109]:
for ix in range(31199,len(combined_df)):
    row = combined_df.iloc[ix]
    url = row['url']
    scrape_url = 'https://'+url
    #print(url,scrape_url)
    res = newspaper_parse(scrape_url)
    #if res[0] is not None:
    title = res[0]
    date = res[1]
    if title is None:
        title = guess_title(scrape_url)
    if date is None:
        date = get_guess_date(scrape_url)
    if title is None and date is None:
        pass
    else:
        json_name = row['guid']
        with open(os.path.join('url_meta',json_name+'.json'),'w') as f:
            json_object = json.dumps({
                    "title":title,
                    "date":date.isoformat() if date is not None else None,
                    "text":res[2]
            }, indent = 4) 
            f.write(json_object)
    if ix % 100 == 0:
        print(ix)
        print(url)

100
newsmax.com/Health/Health-News/death-wish-coffee/2017/09/21/id/814971/
200
newsmax.com/LarryBell/Climate-Change-Global-Warming-U-SState-Facts/2015/11/23/id/703229/
300
newsmax.com/newsfront/france-gas-protests-killed/2018/11/17/id/891059/
400
gizmodo.com/100-websites-that-shaped-the-internet-as-we-know-it-1829634771
500
buzzfeednews.com/article/tomvellner/plants-that-purify-air
600
twincities.com/localnews/ci_27119078/2-environmental-groups-sue-wisconsin-dnr-over-air?source=rss
700
politico.com/tipsheets/politico-influence/2015/12/2015-finale-211922
800
commondreams.org/views/2020/01/30/2020-election-must-focus-voter-needs-and-future?cd-origin=rss
900
feedproxy.google.com/~r/time/politics/~3/FHaXdA9vDW8/0,8599,2057085,00.html
1000
commondreams.org/newswire/2016/01/07/350org-responds-transcanadas-nafta-lawsuit-over-keystone-xl
1100
hosted.ap.org/dynamic/stories/C/CA_CALIFORNIA_INITIATIVE_SPENDING_CAOL-?SITE=CAANR&SECTION=HOME&TEMPLATE=DEFAULT
1200
dailykos.com/stories/2019/8/26/1881

8500
infowars.com/breakthrough-late-night-deal-breaks-two-year-deadlock-over-natural-gas-exports/
8600
blogs.forbes.com/energysource/2011/05/23/british-government-faces-up-to-peak-oil/
8700
nytimes.com/2015/11/20/business/dealbook/buffetts-grandson-seeks-own-investment-route-social-change.html?partner=rss&emc=rss
8800
post-gazette.com/pg/11106/1139688-432-0.stm?cmpid=news.xml
8900
washingtonpost.com/blogs/innovations/wp/2014/04/15/by-2050-every-company-needs-to-be-a-zero-energy-company-or-else/
9000
truthdig.com/articles/california-air-boss-open-to-compromise-in-mileage-standoff/
9100
truthdig.com/articles/california-sets-goal-of-100-percent-clean-energy-by-2045/
9200
sfgate.com/news/article/California-legal-water-cremation-alkaline-burial-12285298.php
9300
seattletimes.com/nation-world/california-says-oceans-expected-to-rise-higher-than-thought/?utm_source=RSS&utm_medium=Referral&utm_campaign=RSS_all
9400
dailycaller.com/2017/04/11/californias-new-fuel-tax-burdens-gas-powered-vehicles

17300
online.wsj.com/article/SB10001424052748703779704576074333552233782.html?mod=fox_australian
17400
dallasnews.com/life/pets/2017/05/26/dog-town-saluting-veterans-four-leggers-things
17500
dailykos.com/stories/2019/7/19/1872968/-Donald-Trump-and-the-GOP-hate-America?utm_campaign=spotlight
17600
motherjones.com/politics/2019/02/donald-trump-still-doesnt-understand-climate-change-science/
17700
theverge.com/2020/3/23/21187076/cruise-line-industry-bailout-trump-coronavirus-us-companies-tax
17800
dotearth.blogs.nytimes.com/2016/11/15/probing-worst-case-environmental-outcomes-under-trump-and-the-g-o-p/?partner=rss&emc=rss
17900
dailykos.com/stories/2018/6/8/1770452/-DrawDown-8-BioPlastic-v-PetroPlastic
18000
huffingtonpost.com/david-h-bailey/dubious-digits-is-this-da_b_6129864.html?utm_hp_ref=science&ir=Science
18100
opednews.com/articles/Earth-Day-Conflict-Over-t-by-Margaret-Flowers-Activism-Environmental_Earth-Day_Energy_Planet-Destruction-180423-347.html
18200
dailykos.com/stories/201

25500
dailycaller.com/2016/04/28/green-activists-push-colorado-to-consider-fracking-ban/
25600
denverpost.com/nationworld/ci_25033471/green-groups-obama-not-doing-enough-climate?source=rss
25700
newsday.com/opinion/commentary/green-new-deal-washington-legislation-economy-energy-1.28770274
25800
buzzfeednews.com/article/danvergano/greenhouse-gas-emissions-are-flattening
25900
infowars.com/greenpeace-founder-no-proof-of-global-warming/
26000
breitbart.com/environment/2019/08/29/greta-thunberg-arrives-nyc-keep-fossil-fuels-ground-stop-war-nature/
26100
breitbart.com/news/grim-tidings-from-science-on-climate-change-2/
26200
staradvertiser.com/2016/09/25/editorial/grow-food-for-hawaii-not-experimental-seed/
26300
feedproxy.google.com/~r/breitbart/~3/CY2wDk3wVBU/
26400
foxnews.com/printer_friendly_wires/2009Mar29/0,4675,BigOilOpportunity,00.html
26500
sacbee.com/news/nation-world/article237706704.html#storylink=rss
26600
washingtontimes.com/news/2017/apr/21/hawaiian-electric-touts-renewable-

KeyboardInterrupt: 

In [111]:
last_ix = 31199

In [112]:
ix

31199